# 🦙 Fully Open-Source RAG in Google Colab


In [3]:
# Uninstall the current PyTorch installation
!pip uninstall torch -y

# Install PyTorch with CUDA 12.1 compatibility
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 706.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 102.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 118.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 573.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.0 MB/s eta 0:00:00
     ━━

In [4]:
# Install required libraries
!pip install -q llama-index-core llama-index-embeddings-huggingface \
                 llama-index-vector-stores-faiss transformers accelerate \
                 torch sentencepiece bitsandbytes

In [5]:
!pip install faiss-gpu-cu12[fix-cuda]


In [6]:
# testing
import faiss
print(faiss.get_num_gpus())

1


In [1]:

# 1️⃣ Imports
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
# 2️⃣ Load your local data (put some .txt files in /content/data)
data_path = "/content/data"
documents = SimpleDirectoryReader(data_path).load_data()

In [3]:
# 3️⃣ Local embedding model (Hugging Face)
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# 4️⃣ Local FAISS vector store
import faiss

# Corrected initialization: Create a FAISS index first
dimension = 384 # This should match the dimension of your embeddings
faiss_index = faiss.IndexFlatL2(dimension) # Example: using L2 distance

faiss_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=faiss_store)

In [9]:
# 5️⃣ Build the vector index
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=embed_model
)

In [ ]:
from huggingface_hub import login

# @title Login to Hugging Face
# @markdown You need to accept the terms and conditions of the model on the Hugging Face website first.
# @markdown You can get a token from your settings page: https://huggingface.co/settings/tokens
try:
  from google.colab import userdata
  HF_TOKEN = userdata.get('HF_TOKEN')
except:
  HF_TOKEN = input("Please enter your Hugging Face token: ")

login(token=HF_TOKEN)

In [7]:
# 6️⃣ Load an open-source LLM (via transformers)
# Recommended: small instruct model to fit in Colab GPU
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_8bit=True  # use less VRAM
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [10]:
# 7️⃣ Query + Generate function (RAG)
def generate_response(query: str):
    # Retrieve top-K context chunks
    retriever = index.as_retriever(similarity_top_k=3)
    retrieved_docs = retriever.retrieve(query)
    context_text = "\n\n".join([d.get_text() for d in retrieved_docs])

    # Build final prompt for LLM
    prompt = (
        f"Context:\n{context_text}\n\n"
        f"Question: {query}\n\n"
        f"Answer concisely using the context above."
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=256)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [11]:
# 8️⃣ Example usage
query = "what are the skills of the person in given resume"
print(generate_response(query))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Context:
is the Linkedin profile link)
  /K [<<
  /Type /MCR
  /Pg 36 0 R
  /MCID 5
>> <<
  /Type /MCR
  /Pg 36 0 R
  /MCID 6
>> 21 0 R <<
  /Type /OBJR
  /Pg 36 0 R
  /Obj 10 0 R
>> <<
  /Type /OBJR
  /Pg 36 0 R
  /Obj 11 0 R
>>]
>>
endobj
21 0 obj
<<
  /S /Span
  /P 20 0 R
  /K [<<
  /Type /MCR
  /Pg 36 0 R
  /MCID 7
>>]
>>
endobj
107 0 obj
<<
  /S /P
  /P 104 0 R
  /K [109 0 R]
>>
endobj
109 0 obj
<<
  /S /Link
  /P 107 0 R
  /Alt (This is the website link)
  /K [22 0 R <<
  /Type /OBJR
  /Pg 36 0 R
  /Obj 13 0 R
>>]
>>
endobj
22 0 obj
<<
  /S /Span
  /P 109 0 R
  /K [<<
  /Type /MCR
  /Pg 36 0 R
  /MCID 8
>> 23 0 R]
>>
endobj
23 0 obj
<<
  /S /Span
  /P 22 0 R
  /K [<<
  /Type /MCR
  /Pg 36 0 R
  /MCID 9
>>]
>>
endobj
108 0 obj
<<
  /S /Div
  /P 104 0 R
  /K [24 0 R 110 0 R]
>>
endobj
24 0 obj
<<
  /S /H1
  /P 108 0 R
  /K [<<
  /Type /MCR
  /Pg 36 0 R
  /MCID 10
>>]
>>
endobj
110 0 obj
<<
  /S /Div
  /P 108 0 R
  /K [111 0 R 112 0 R 113 0 R]
>>
endobj
111 0 obj
<<
  /S /P
  /P 110